In [19]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime

In [20]:
LIMIT = 4

data = {}
data['newspapers'] = {}

# Loads the JSON files with news sites
with open(r'C:\Users\Sony\Desktop\news.json') as data_file:
    companies = json.load(data_file)

In [21]:
count = 1

In [22]:
# Iterate through each news company
for company, value in companies.items():
    # If a RSS link is provided in the JSON file, this will be the first choice.
    # Reason for this is that, RSS feeds often give more consistent and correct data.
    # If you do not want to scrape from the RSS-feed, just leave the RSS attr empty in the JSON file.
    if 'rss' in value:
        d = fp.parse(value['rss'])
        print("Downloading articles from ", company)
        newsPaper = {
            "rss": value['rss'],
            "link": value['link'],
            "articles": []
        }
        for entry in d.entries:
            # Check if publish date is provided, if no the article is skipped.
            # This is done to keep consistency in the data and to keep the script from crashing.
            if hasattr(entry, 'published'):
                if count > LIMIT:
                    break
                article = {}
                article['link'] = entry.link
                date = entry.published_parsed
                article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                except Exception as e:
                    # If the download for some reason fails (ex. 404) the script will continue downloading
                    # the next article.
                    print(e)
                    print("continuing...")
                    continue
                article['title'] = content.title
            
                newsPaper['articles'].append(article)
                print(count, "articles downloaded from", company, ", url: ", entry.link)
                count = count + 1
    else:
        # This is the fallback method if a RSS-feed link is not provided.
        # It uses the python newspaper library to extract articles
        print("Building site for ", company)
        paper = newspaper.build(value['link'], memoize_articles=False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            # Again, for consistency, if there is no found publish date the article will be skipped.
            # After 10 downloaded articles from the same newspaper without publish date, the company will be skipped.
            if content.publish_date is None:
                print(count, " Article has date of type None...")
                noneTypeCount = noneTypeCount + 1
                if noneTypeCount > 10:
                    print("Too many noneType dates, aborting...")
                    noneTypeCount = 0
                    break
                count = count + 1
                continue
            article = {}
            article['title'] = content.title
            
            article['link'] = content.url
            article['published'] = content.publish_date.isoformat()
            newsPaper['articles'].append(article)
            print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            noneTypeCount = 0
    count = 1
    data['newspapers'][company] = newsPaper

Building site for  Hindustan Times
1 articles downloaded from Hindustan Times  using newspaper, url:  https://www.hindustantimes.com/lok-sabha-elections/2019-lok-sabha-elections-ncp-chief-sharad-pawar-is-no-loser-in-maharashtra/story-xO04WYvyjAaPLKU4JPUBzH.html
2 articles downloaded from Hindustan Times  using newspaper, url:  https://www.hindustantimes.com/opinion/lok-sabha-elections-2019-will-the-triangular-contest-work-to-bjp-s-advantage-in-uttar-pradesh/story-TKcULaIj24tFx6zfzYocSN.html
3 articles downloaded from Hindustan Times  using newspaper, url:  https://www.hindustantimes.com/lok-sabha-elections/lok-sabha-elections-2019-bjp-s-last-minute-move-wins-over-disgruntled-allies/story-t65vchMTSGkvelYBJV3gHK.html
4 articles downloaded from Hindustan Times  using newspaper, url:  https://www.hindustantimes.com/lok-sabha-elections/lok-sabha-elections-2019-bjp-to-reach-out-to-minorities-dalits-in-uttarakhand-as-part-of-poll-strategy/story-PRaw1bT8ZliIFTgFaAmLhL.html
1 articles downloade

In [27]:
try:
    with open(r'C:\Users\Sony\Desktop\nwsppr1.xls', 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)

# If you want in json format

In [ ]:
try:
    with open(r'C:\Users\Sony\Desktop\nws.json', 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)